In [1]:
import sys
sys.path.insert(0,'/home/djl34/kl_git/scripts')
from notebook_imports import *

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
client = Client()

In [3]:
rate = os.path.join(scratch_dir, "results/footprints/denovo/{chrom}.tsv").format(chrom = 22)
neutral = os.path.join(KL_data_dir, "whole_genome/freq_bins/freq_bin_9_all.tsv")

ddf = dd.read_csv(rate, sep = "\t", dtype={'Spliceai_info': 'object'})

freq_breaks_9 = [-1, 1e-8, 2e-06, 4.0e-06, 8.0e-06, 1.6e-05, 5e-05, 5e-04, 5e-03, 0.5] 
ddf["Freq_bin_9"] = ddf['MAF'].map_partitions(pd.cut, freq_breaks_9, labels = False)

neutral = pd.read_csv(neutral, sep = "\t")

In [4]:
neutral_sfs = neutral.drop(columns = ["mu", "sum"]).to_numpy()

neutral_sfs_beta = simt.multinomial_trans(neutral_sfs)

In [43]:
neutral_sfs

array([[9.74654952e-01, 1.44045729e-02, 3.45998907e-03, 2.56284214e-03,
        1.75868193e-03, 1.61197292e-03, 8.93950992e-04, 2.96461786e-04,
        3.56575958e-04],
       [9.60506883e-01, 2.21025916e-02, 5.46049124e-03, 4.11714399e-03,
        2.76669764e-03, 2.57111077e-03, 1.48175864e-03, 4.93494452e-04,
        4.99828389e-04],
       [9.43247293e-01, 3.14031691e-02, 7.99858947e-03, 5.99032430e-03,
        4.04536634e-03, 3.75720938e-03, 2.15823269e-03, 7.20392405e-04,
        6.79422943e-04],
       [9.24831252e-01, 4.11243766e-02, 1.07270441e-02, 8.04825732e-03,
        5.40666756e-03, 5.10931741e-03, 2.90829380e-03, 9.55300419e-04,
        8.89490333e-04],
       [9.05610121e-01, 5.10926965e-02, 1.35893915e-02, 1.02259192e-02,
        6.95169776e-03, 6.43652376e-03, 3.70695545e-03, 1.23294109e-03,
        1.15375392e-03],
       [8.86632990e-01, 6.06737059e-02, 1.65111669e-02, 1.24407379e-02,
        8.45977621e-03, 7.86128929e-03, 4.52095588e-03, 1.48784222e-03,
        1.4

In [6]:
type(neutral_sfs)

numpy.ndarray

In [7]:
array = np.zeros(neutral_sfs.shape)


In [8]:
array[0,2] += 1

In [13]:
array

array([[0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0., 0., 0., 0.],
       [4., 1., 0., 0., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0.

In [10]:
## create some function that groupby and then transforms into array

ddf_footprint = ddf[ddf["footprint_identifier"] == "22.38562.4"].compute()

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [11]:
array_positions = pd.DataFrame(ddf_footprint.groupby(["mu_index", "Freq_bin_9"]).size()).reset_index()

In [12]:
for index, row in array_positions.iterrows():
    array[row['mu_index'], row['Freq_bin_9']] += row[0]

In [17]:
array

array([[0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0., 0., 0., 0.],
       [4., 1., 0., 0., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0.

In [37]:
ddf_footprint[ddf_footprint["Freq_bin_9"] == 4]

,index,Pos,mu,Allele_ref,Allele,mu_index,AN_total_interpolate,AC_total,Neutral,AF,MAF,exon,Spliceai_info,DHS_mean_signal,DHS_sample,DHS_identifier,DHS_tissue,DHS_core_mean_signal,footprint_mean_signal,footprint_samples,footprint_motif_clusters,footprint_identifier,footprints,denovo_an_case,denovo_an_control,Freq_bin_9
102632,16146004_C_G,16146004,0.062,C,G,5,552196.0,5.0,NaN,0.000009,0.000009,False,NaN,0.957756,5.0,22.38562,Renal / cancer,NaN,6.467998,1.0,NaN,22.38562.4,1,NaN,NaN,4


In [14]:
def multiply(x):
    return x[0]*x[1]

In [36]:
# Create a WinSFS object and fit the SFS using maximum likelihood
#add pseudocount
pseudocount_array = array + 10 * neutral_sfs
winsfs_test = mlr.WinSFS(data = pseudocount_array, mut_offset = neutral_sfs_beta)

In [37]:
winsfs_test.ml_optim(jac=True, beta_max=100, verbose=False)

In [40]:
for i in range(10):
    print(winsfs_test.KL(i))

0.0
0.0
0.0
-2.372997710928137e-16
0.0
0.0
-2.5641911988387646e-16
-1.1793928511207434e-16
0.0
0.0


In [42]:
pseudocount_array

array([[9.74654952e+00, 1.44045729e-01, 1.03459989e+00, 2.56284214e-02,
        1.75868193e-02, 1.61197292e-02, 8.93950992e-03, 2.96461786e-03,
        3.56575958e-03],
       [1.16050688e+01, 2.21025916e-01, 5.46049124e-02, 4.11714399e-02,
        2.76669764e-02, 2.57111077e-02, 1.48175864e-02, 4.93494452e-03,
        4.99828389e-03],
       [1.34324729e+01, 3.14031691e-01, 7.99858947e-02, 5.99032430e-02,
        4.04536634e-02, 3.75720938e-02, 2.15823269e-02, 7.20392405e-03,
        6.79422943e-03],
       [1.32483125e+01, 1.41124377e+00, 1.07270441e-01, 8.04825732e-02,
        5.40666756e-02, 5.10931741e-02, 2.90829380e-02, 9.55300419e-03,
        8.89490333e-03],
       [1.10561012e+01, 5.10926965e-01, 1.35893915e-01, 1.02259192e-01,
        6.95169776e-02, 6.43652376e-02, 3.70695545e-02, 1.23294109e-02,
        1.15375392e-02],
       [9.86632990e+00, 6.06737059e-01, 1.65111669e-01, 1.24407379e-01,
        1.08459776e+00, 7.86128929e-02, 4.52095588e-02, 1.48784222e-02,
        1.4

In [44]:
neutral_sfs * 10

array([[9.74654952e+00, 1.44045729e-01, 3.45998907e-02, 2.56284214e-02,
        1.75868193e-02, 1.61197292e-02, 8.93950992e-03, 2.96461786e-03,
        3.56575958e-03],
       [9.60506883e+00, 2.21025916e-01, 5.46049124e-02, 4.11714399e-02,
        2.76669764e-02, 2.57111077e-02, 1.48175864e-02, 4.93494452e-03,
        4.99828389e-03],
       [9.43247293e+00, 3.14031691e-01, 7.99858947e-02, 5.99032430e-02,
        4.04536634e-02, 3.75720938e-02, 2.15823269e-02, 7.20392405e-03,
        6.79422943e-03],
       [9.24831252e+00, 4.11243766e-01, 1.07270441e-01, 8.04825732e-02,
        5.40666756e-02, 5.10931741e-02, 2.90829380e-02, 9.55300419e-03,
        8.89490333e-03],
       [9.05610121e+00, 5.10926965e-01, 1.35893915e-01, 1.02259192e-01,
        6.95169776e-02, 6.43652376e-02, 3.70695545e-02, 1.23294109e-02,
        1.15375392e-02],
       [8.86632990e+00, 6.06737059e-01, 1.65111669e-01, 1.24407379e-01,
        8.45977621e-02, 7.86128929e-02, 4.52095588e-02, 1.48784222e-02,
        1.4